In [ ]:
#Import Libraries and Load Raw Dato
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from scipy.integrate import quad
from scipy import stats, interpolate
import warnings
import os
#Loads Data 
max_temp = pd.read_csv('maximum_temperature.csv')
min_temp = pd.read_csv('minimum_temperature.csv')
max_temp.head()

#Clean dat of Null Vaues and Show it is quantity
max_temp.isnull().value_counts(),min_temp.isna().value_counts
count = 0
for mx, mn in zip(np.where(max_temp.isnull())[0], np.where(min_temp.isnull())[0]):
    if mx != mn:
        count +=1
print('Number of Null Values: ', count)


#Fram raw Data, Extracts Date (Y-M-D) Maximum and Minimum Temperatures (Tmax, and Tmin) and calculate the Daily Average ( T=(Tmax+Tmin)/2)


def datetime(row):
    return dt.datetime(row.Year, row.Month, row.Day)
max_temp['Date'] = max_temp.apply(datetime,axis=1)
min_temp['Date'] = min_temp.apply(datetime,axis=1)
max_temp.set_index('Date',inplace=True)
min_temp.set_index('Date', inplace=True)

#MDrops column with useless data, renames the columns left, and merges the data frames
drop_cols =[0,1,2,3,4,6,7]
max_temp.drop(max_temp.columns[drop_cols], axis=1, inplace=True)
min_temp.drop(min_temp.columns[drop_cols], axis=1, inplace=True)
max_temp.rename(columns={'Maximum temperature (Degree F)': 'Tmax'}, inplace=True)
max_temp.rename(columns={'Minimum temperature (Degree F)': 'Tmin'}, inplace=True)
temps = max_temp.merge(min_temp, how=' inner',left_on=['Date'],right_on=['Date'])
#Calculates Daily Average droping null values.

def avg_temp(row):
    return (row.Tmax+row.Tmin)/2
temps['T'] = temps.apply(avg_temp, axis=1)
# drop na values here 
temps  = temps.dropna()

temps_season = temps.copy(deep=True)
temp_t = temps['T'].copy(deep=True)
temp_t = temp_t.to_frame()
first_ord = temp_t.index.map(dt.datetime.toordinal)[0]

#Adds a month and day columm to the data
temp_vol = temps['T'].copy(deep=True).to_frame()
temp_vol['day'] = temp_vol.index.dayofyear
temp_vol['month'] = temp_vol.index.month

#Constructs time series with volatility
vol = temp_vol.groupby(['day'])['T'].agg(['mean','std'])
days = np.array(vol['std'].index)
T_std = np.array(vol['std'].values)
temp_t.head()

#Uses B-Spline and Interpolates DAT data with B-Spline and Calculates Ornstein-Uhlnebeck termsModel with Previously obtained parametes
#T = a+bx+a1sin((omega)x+Theta)+b1cos((omega)x+phi)
Tbar_params = [62.1, 6.08e-05, 9.03, 1.26]
def T_model(x, a, b, alpha, theta):
    omega = 2*np.pi/365.25
    T = a + b*x + alpha*np.sin(omega*x + theta)
    return T
def dT_model(x, a, b, alpha, theta):
    omega=2*np.pi/365.25
    dT = b + alpha*omega*np.cos(omega*x + theta)
    return dT
def spline(knots, x, y):
    x_new = np.linspace(0, 1, knots+2)[1:-1]
    t, c, k = interpolate.splrep(x, y, t=np.quantile(x, x_new), s=3)
    yfit = interpolate.BSpline(t,c, k)(x)
    return yfit
#Apply Euler approximation to Time Series
def euler_step(row, kappa, M, lamda):
    #*** Function for Euler sheme approximation step in
    #midified or Dynamics for temperature simulations
    #inputs:
    #dataframe row with columns: T, Tbar, dTbar and vol
    #kappa: rate of mean reversion from data processing code
    #Output:
    # temp:simulated next day temperature
    if row['T'] != np.nan:
        T_i = row['Tbar']
    else:
        T_i = row['T']
    T_det = T_i + row['dTbar']
    T_mrev = kappa*(row['Tbar'] - T_i)
    sigma = row['vol']*np.random.randn(M)
    riskn = lamda*row['vol']
    return T_det + T_mrev + sigma - riskn

#Apply Monte Carlo approximation to time series
def monte_carlo_temp(trading_dates, Tbar_params, vol_model, first_ord, M=1, lamda=0):
    #Monte Carlo simulation of temperature
    #Inputs:
    #trading_dates: pandas DatetimeIndex from start to end dates
    #M:number of simulations
    #Tbar_params:parameters used for Tbar model
    #vol_model: fitted volatility model with days in year index
    #first_ord: first ordinal of fitted Tbar model
    # Output:
    # mc_temps: DataFrame of all components and simulated temperatures
    kappa=0.438
    if isinstance(trading_dates, pd.DatetimeIndex):
        trading_date=trading_dates.map(dt.datetime.toordinal)
    Tbars = T_model(trading_date-first_ord, *Tbar_params)
    dTbars = dT_model(trading_date-first_ord, *Tbar_params)
    mc_temps = pd.DataFrame(data=np.array([Tbars, dTbars]).T, index=trading_dates, columns=['Tbar', 'dTbar'])
    mc_temps['day'] = mc_temps.index.dayofyear
    mc_temps['vol'] =vol_model[mc_temps['day']-1]

    mc_temps['T'] =mc_temps['Tbar'].shift(1)
    data = mc_temps.apply(euler_step, args=[kappa, M, lamda], axis=1)
    mc_sims = pd.DataFrame(data=[x for x in [y for y in data.values]], index=trading_dates, columns=range(1, M+1))
    return mc_temps, mc_sims
# Define trading date range
S='2024-05-01'
E='2024-11-01'
trading_dates = pd.date_range(start=S, end= E, freq='D')
volatility = spline(5, days, T_std)
mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params, volatility, first_ord)
plt.figure(figsize=(10,6))
mc_sims[1].plot(alpha=0.5, linewidth=1, marker='.')
mc_temps["Tbar"].plot(linewidth=3)
plt.legend(loc='lower right')
plt.show()
#Calculate probability of no payoff within time range
trading_dates = pd.date_range(start = S, end = E, freq='D')
volatility = spline(5, days, T_std)
mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params, volatility, first_ord)
#Necessita converter Celsius usar 18 C
print('Probability P(max(65-tn, 0) 0): {0:1.1f}%'. format(len(mc_sims[mc_sims[1] >=65])/len(mc_sims)*100))
def rn_mean(time_arr, vol_arr, Tbars, lamda, kappa):
    #Evaluate the risk neutral integral above for each segment of constant volatility
    #Rectangular integration with step size of days
    dt = 1/365.25
    N = len(time_arr)
    mean_intervals = -vol_arr*(1 - np.exp(-kappa*dt))/kappa
    return 65*N - (np.sum(Tbars) - lamda*np.sum(mean_intervals))

def rn_var(time_arr, vol_arr, kappa):
    #Evaluate the risk neutral integral above for each segmant of constant volatility
    #Rectangular integration with size of days
    dt = 1/365.25
    var_arr = np.power(vol_arr,2)
    var_intervals = var_arr/(2*kappa)*(1-np.exp(-2*kappa*dt))
    cov_sum = 0
    for i, ti in enumerate(time_arr):
        for j, tj in enumerate(time_arr):
            if j > i:
                cov_sum += np.exp(-kappa*(tj-ti)) * var_intervals[i]

    return np.sum(var_intervals) + 2*cov_sum
def risk_neutral(trading_dates, Tbar_params, vol_model, first_ord, lamda, kappa=0.438):
    if isinstance(trading_dates, pd.DatetimeIndex):
        trading_date=trading_dates.map(dt.datetime.toordinal)

    Tbars = T_model(trading_date-first_ord, *Tbar_params)
    dTbars = dT_model(trading_date-first_ord, *Tbar_params)
    mc_temps = pd.DataFrame(data=np.array([Tbars, dTbars]).T, index=trading_dates, columns=['Tbar', 'dTbar'])
    mc_temps['day'] = mc_temps.index.dayofyear
    mc_temps['vol'] = vol_model[mc_temps['day']-1]
    time_arr = np.array([i/365.25 for i in range(1,len(trading_dates)+1)])
    vol_arr = mc_temps['vol'].values
    mu_rn = rn_mean(time_arr, vol_arr, Tbars, lamda, kappa)
    var_rn = rn_var(time_arr, vol_arr, kappa)
    return mu_rn, var_rn
def winter_option(trading_dates, r, alpha, K, tau, opt='c', lamda=0.0):
    #Evaluate the fair value of temperature option in winter
    #Based on heating degree days only if the physical probability that
    #the average temperature exceeds the Tref(65 deg F or 18 C) is close to zero
    mu_rn, var_rn = risk_neutral(trading_dates, Tbar_params, volatility, first_ord, lamda)
    disc = np.exp(-r*tau)
    vol_rn = np.sqrt(var_rn)
    zt = (K - mu_rn)/vol_rn
    exp = np.exp(-zt**2/2)
    if opt == 'c':
        return alpha*disc*((mu_rn - K)*stats.norm.cdf(-zt) + vol_rn*exp/np.sqrt(2*np.pi))
    else:
        exp2 = np.exp(-mu_rn**2/(2*vol_rn**2))
        return alpha*disc*((K - mu_rn)*(stats.norm.cdf(zt) - stats.norm.cdf(-mu_rn/vol_rn)) + vol_rn/np.sqrt(2*np.pi)*(exp-exp2))
trading_dates = pd.date_range(start= S, end= E, freq ='D')
r=0.05
K=300
alpha=2500
def years_between(d1, d2):
    d1 = dt.datetime.strptime(d1, "%Y-%m-%d")
    d2 = dt.datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)/365.25
#Remember change date today()
start = dt.datetime.today().strftime('%Y-%m-%d')
end = E
tau = years_between(start, end)
print('Start Valuation date: ', start, '\nEnd of Contract Date:', end, '\nYears between Dates:', round(tau,3))
print('Call Price :', round (winter_option(trading_dates, r, alpha, K, tau, 'c'),2))
print('Put Price  :', round (winter_option(trading_dates, r, alpha, K, tau, 'p'),2))


#define trading date range
trading_dates = pd.date_range(start = S, end= E, freq='D')
no_sims = 10000
vol_model = spline(5, days, T_std)
def temperature_option(trading_dates, no_sims, Tbar_params, vol_model, r, alpha, K, tau, first_ord, opt='c'):
    "Evaluates the price of a temperature of call option"
    mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params, volatility, first_ord, no_sims)
    N, M = np.shape(mc_sims)
    mc_arr = mc_sims.values
    D0 = np.sum(np.maximum(mc_arr-65,0), axis=0)
    if opt == 'c':
        CT = alpha*np.maximum(D0-K,0)
    else:
         CT = alpha*np.maximum(K-D0,0)
    C0 = np.exp(-r*tau)*np.sum(CT)/M
    sigma = np.sqrt(np.sum( (np.exp(-r*tau)*CT - C0)**2)/(M-1))
    SE = sigma/np.sqrt(M)
    return C0, SE
call = np.round(temperature_option(trading_dates, no_sims, Tbar_params, vol_model, r, alpha, K, tau, first_ord, 'c'),2)
put = np.round(temperature_option(trading_dates, no_sims, Tbar_params, vol_model, r, alpha, K, tau, first_ord, 'p'),2)
print('Call Price: {0} +/- {1} (2se)'.format(call[0], call[1]*2))
print('Put Price: {0} +/- {1} (2se)'.format(call[0], call[1]*2))
#Compare Black Scholes with MC Simulations Method
strikes = np.arange(400,1600,50)
data = np.zeros(shape=(len(strikes),4))
for i, strike in enumerate(strikes):
    data[i,0] = temperature_option(trading_dates, no_sims, Tbar_params, vol_model, r, alpha, strike, tau, first_ord, 'c')[0]
    data[i,1] = winter_option(trading_dates, r, alpha, strike, tau, 'c')
    data[i,2] = temperature_option(trading_dates, no_sims, Tbar_params, vol_model, r, alpha, strike, tau, first_ord, 'p')[0]
    data[i,3] = winter_option(trading_dates, r, alpha, strike, tau, 'p')
df = pd.Dataframe({'MC Call': data[:, 0], 'BSA Call': data[:, 1], 'MC Put': data[:,2], 'BSA Put': data[:, 3]})
df.index = strikes
plt.plot(df)
plt.title('Winter Temperature Options')
plt.ylabel('Option Price USD')
plt.xlabel('Strikes (HDD)') # Alterar para CDD
plt.legend(df.columns, loc=4)
plt.show()












    





















